In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install contractions

In [ ]:
!pip install spacy-transformers

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
!pip install spacy==3.7.4

In [ ]:
!pip install node2vec

  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1


restart runtime

In [26]:
import os
import pandas as pd
import numpy as np
import spacy
import re
import contractions
import subprocess
from pathlib import Path
from textblob import TextBlob
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Bidirectional, LSTM
import joblib
from spacy.lang.en import stop_words
stop_words = list(stop_words.STOP_WORDS)
nlp = spacy.load('en_core_web_sm')
import spacy_transformers
bert_sent = spacy.load("en_core_web_trf")


import networkx as nx
import matplotlib.pyplot as plt
#from nltk.parse.corenlp import CoreNLPDependencyParser
#dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')
from node2vec import Node2Vec

In [ ]:
try:
  os.mkdir('/content/drive/MyDrive/DATA')
except:
  res = 0
!tar -xf "fns2020_dataset.tar.gz" -C "/content/drive/MyDrive/DATA"
!tar -xf "/content/drive/MyDrive/DATA/training.tar.gz" -C "/content/drive/MyDrive/DATA/"
!tar -xf "/content/drive/MyDrive/DATA/validation.tar.gz" -C "/content/drive/MyDrive/DATA/"
os.remove("/content/drive/MyDrive/DATA/training.tar.gz")
os.remove("/content/drive/MyDrive/DATA/validation.tar.gz")

In [ ]:
def ten_percent(path):
  folders = ['training/annual_reports', 'validation/annual_reports']
  for folder in folders:
    path_to_folder1 = os.path.join(path, folder)
    content_folder = os.listdir(path_to_folder1)
    k = 0
    for ar_file_name in content_folder:
      if ar_file_name.split('.')[-1] == 'csv':
        continue
      print("remaining : "+str(len(content_folder)-k))
      k += 1
      path_to_file1 = os.path.join(path_to_folder1, ar_file_name)
      text_ar = open(path_to_file1)
      text = text_ar.read()
      try:
        doc = nlp(text)
      except:
        res = 0

      sentences = list(doc.sents)
      n = int(len(sentences)/10)
      cleaned_sentences = []
      for sent in sentences[:n]:
        sent = str(sent)
        cleaned_sentences.append(sent)
      data = {"sentences": cleaned_sentences}
      df = pd.DataFrame(data)
      name = ar_file_name[:-3]+'csv'
      path_to_file2 = os.path.join(path_to_folder1, name)
      try:
        df.to_csv(path_to_file2)
        os.remove(path_to_file1)
      except:
        res = 0
        os.remove(path_to_file1)
  return 0

ten_percent('/content/drive/MyDrive/DATA')

In [ ]:
def remove_telephone_numbers(sentence):
  i = 0
  remove = []
  sentence = str(sentence)
  while i < len(sentence):
    if sentence[i] == '+':
      count = 0
      j = i+1
      while j < len(sentence):
        if sentence[j] >='0' and sentence[j] <='9':
          count += 1
          j += 1
        elif sentence[j] == ' ' or sentence[j] == '(' or sentence[j] == ')':
          j += 1
        else:
          break
      print(count)
      if count >= 10 and count <= 13:
        sentence = sentence.replace(sentence[i:j], "")
        i = -1
    i = i+1
  return sentence

def remove_addresses_sent(text):
  text = str(text)
  clean_text = re.sub(r"\S*www.\S+", "", text)
  clean_text = re.sub(r"\S*WWW.\S+", "", clean_text)
  clean_text = re.sub(r"\S*.com\S+", "", clean_text)
  clean_text = re.sub(r"\S*.co\S+", "", clean_text)
  clean_text = re.sub(r"\S*.COM\S+", "", clean_text)
  clean_text = re.sub(r"\S*.gov\S+", "", clean_text)
  clean_text = re.sub(r"\S*.biz\S+", "", clean_text)
  clean_text = re.sub(r"\S*.org\S+", "", clean_text)
  clean_text = re.sub(r"\S*.uk\S+", "", clean_text)
  return clean_text

def preprocess_text(sentence):
    # Remove punctuation characters
    sentence = str(sentence)
    sentence = re.sub(r"[^a-zA-Z0-9]", " ", sentence)

    # Tokenization
    tokens = nlp(sentence)

    # Remove stopwords
    lemmatized = []
    for token in tokens:
      token_lemma= str(token.lemma_)
      token_lemma = token_lemma.lower()
      if token_lemma not in stop_words and token_lemma[0] != ' ':
        lemmatized.append(token_lemma)
    return lemmatized

In [ ]:
def labelling(path):
  folders1 = ['training/annual_reports', 'validation/annual_reports']
  folders2 = ['training/gold_summaries', 'validation/gold_summaries']
  for i in range(len(folders1)):
    folder = folders1[i]
    path1 = os.path.join(path, folder)
    contents1 = os.listdir(path1)
    for ar_name in contents1:
      path2 = os.path.join(path1, ar_name)
      df1 = pd.read_csv(path2)
      sentences = list(df1['sentences'])
      labels = []
      compare_sentences = []
      path3 = os.path.join(path, folders2[i])
      gs_files = os.listdir(path3)
      for x in gs_files:
        if x.split('_')[0] == ar_name.split('.')[0]:
          path4 = os.path.join(path3, x)
          file_pt = open(path4)
          text = file_pt.read()
          doc = nlp(text)
          for sen in list(doc.sents):
            compare_sentences.append(str(sen))
      compare_sentences = list(set(compare_sentences))
      #print(compare_sentences[0])
      cleaned_sentences = []
      for x in sentences:
        sent = remove_telephone_numbers(x)
        sent = remove_addresses_sent(sent)
        expanded_text = []
        for word in sent.split(' '):
          expanded_text.append(contractions.fix(word))
        expanded_text = ' '.join(expanded_text)
        sent_cleaned = preprocess_text(expanded_text)
        final_form = ' '.join(sent_cleaned)
        cleaned_sentences.append(final_form)
        if x in compare_sentences:
          labels.append(1)
        else:
          labels.append(0)
      df1['cleaned'] = cleaned_sentences
      df1['label'] = labels
      df1.to_csv(path2)

labelling('/content/drive/MyDrive/DATA')

Calculating tf score of n-grams(<=3)

In [ ]:
def calculate_IDF(freq_dict, doc):
    unique_words = list(set(doc))
    for word in unique_words:
        if word in freq_dict:
            freq_dict[word] += 1
        else:
            freq_dict[word] = 1
    return freq_dict

def wrapper_calculate_IDF(path_to_training_data, n_gram):
    freq_dict = {}
    ars = os.listdir(path_to_training_data)
    files_remaining = 0
    for ar in ars:
        #print(len(ars)-files_remaining)
        files_remaining += 1
        path = os.path.join(path_to_training_data, ar)
        sentences_df = pd.read_csv(path)
        sentences = list(sentences_df["cleaned"])
        cleaned_sentences = [] #as list of words
        for sent in sentences:
            sent = str(sent)
            sent = sent.split(' ')
            cleaned_sentences.append(sent)

        n_gram_of_sentences = []
        for n_gram_count in range(n_gram):
            temp = []
            for sent in cleaned_sentences:
                for i in range(len(sent)):
                    j = i+n_gram_count+1
                    if j <= len(sent):
                        n_gram_temp = ' '.join(sent[i:j])
                        temp.append(n_gram_temp)
            n_gram_of_sentences.extend(temp)
        freq_dict = calculate_IDF(freq_dict, n_gram_of_sentences)
        #print(freq_dict)
    return freq_dict, len(ars)

def save_idf(path_to_training_data):
    idf_dict, n = wrapper_calculate_IDF(path_to_training_data, n_gram=3)
    token_list = list(idf_dict.keys())
    N_list = []
    for i in range(len(token_list)):
        N_list.append(n)
    idf_freq_raw = list(idf_dict.values())
    idf_freq = np.divide(N_list, idf_freq_raw)
    idf_freq = np.log(idf_freq)
    data = {"tokens": token_list, "idf_frequency": idf_freq_raw, "total doucuments":N_list, "idf":idf_freq}
    df = pd.DataFrame(data)
    df_path = path_to_training_data+'_IDF.csv'
    print(df_path)
    df.to_csv(df_path)

def main1(path):
  folders = ['training/annual_reports', 'validation/annual_reports']
  for folder in folders:
    path2 = os.path.join(path, folder)
    save_idf(path2)
main1('/content/drive/MyDrive/DATA')

/content/drive/MyDrive/DATA/training/annual_reports_IDF.csv
/content/drive/MyDrive/DATA/validation/annual_reports_IDF.csv


Calculating tfidf scores of ngrams(<=3)

In [ ]:
def calc_tfidf(path, n_gram, idf_dict):
  contents = os.listdir(path)
  remain = 0
  for ar in contents:
    #print(len(contents)-remain)
    remain += 1
    tf_dict = {}
    path1 = os.path.join(path, ar)
    sentences_df = pd.read_csv(path1)
    sentences = list(sentences_df["cleaned"])

    n_gram_of_sentences = []

    for n_gram_count in range(n_gram):
      temp = []
      for sent in sentences:
        sent = str(sent)
        sent = sent.split(' ')
        for i in range(len(sent)):
          j = i+n_gram_count+1
          if j <= len(sent):
            n_gram_temp = ' '.join(sent[i:j])
            temp.append(n_gram_temp)
      if n_gram_count == 0:
        doc_length = float(len(temp))
      n_gram_of_sentences.extend(temp)

    for token in n_gram_of_sentences:
      if token in tf_dict:
        tf_dict[token] += 1
      else:
        tf_dict[token] = 1

    keys = tf_dict.keys()
    for key in keys:
      val = tf_dict[key]
      T_len = len(key.split(' '))
      tf_dict[key] = val/float(doc_length-T_len+1)

    tf_idf_scores = []
    if len(sentences)==0:
      os.remove(path1)
      continue
    for sent in sentences:
      sent = str(sent)
      sent = sent.split(' ')
      empty_indices = 0
      for l in range(len(sent)):
        if sent[l] == '':
          empty_indices += 1
      for l in range(empty_indices):
        sent.remove('')

      temp = []
      for n_gram_count in range(n_gram):
        for i in range(len(sent)):
          j = i+n_gram_count+1
          if j <= len(sent):
            n_gram_temp = ' '.join(sent[i:j])
            temp.append(n_gram_temp)
      tf_idf_score = 0
      for x in temp:
        try:
          score_temp = idf_dict[x]
        except:
          score_temp = 0
        score_temp *= tf_dict[x]
        tf_idf_score += score_temp
      tf_idf_scores.append(tf_idf_score)
    max_val = np.max(tf_idf_scores)
    tf_idf_scores = np.divide(tf_idf_scores, max_val)
    #print(tf_idf_scores)
    sentences_df["tf_idf_n_gram_"+str(n_gram)] = np.array(tf_idf_scores)
    sentences_df.to_csv(path1)


def main2(path, n_gram = 3):
  folders = ['training/annual_reports', 'validation/annual_reports']
  for folder in folders:
    path1 = os.path.join(path, folder)
    idf_df = pd.read_csv(path1+'_IDF.csv')
    tokens = idf_df["tokens"]
    idf_score = idf_df["idf"]
    idf_dict = {}
    for i in range(len(tokens)):
      idf_dict[tokens[i]] = idf_score[i]
    for i in range(n_gram):
      calc_tfidf(path1, i+1, idf_dict)
main2('/content/drive/MyDrive/DATA')

Sentence Bert Embeddings

In [ ]:
def bert_embedding_generate(path, path_bert):
  contents = os.listdir(path)
  l = 0
  for ar in contents:
    print("remaining : "+str(len(contents)- l))
    l += 1
    path1 = os.path.join(path, ar)
    df = pd.read_csv(path1)
    sentences = list(df['sentences'])

    embedding = []
    for sent in sentences:
      doc = bert_sent(sent)
      tokvecs = doc._.trf_data.last_hidden_layer_state[0]
      embed = np.mean(tokvecs.data, axis=0)
      embed = list(embed)

      sentiment_score = TextBlob(sent)
      embed.extend([sentiment_score.sentiment[0], sentiment_score.sentiment[1]])
      embedding.append(embed)

    name = ar[:-4]+'.npy'
    path_to_npy_file = os.path.join(path_bert, name)
    with open(path_to_npy_file, 'wb') as f:
      np.save(f, np.array(embedding))

def main3(path):
  folders = ['training/annual_reports', 'validation/annual_reports']
  for folder in folders:
    path1 = os.path.join(path, folder)
    path2 = os.path.join(path, folder+'_bert_embedding')
    try:
      os.mkdir(path2)
      print(path2)
    except:
      res = 0

    bert_embedding_generate(path1, path2)
main3('/content/drive/MyDrive/DATA')

Sentence Node Embeddings

In [ ]:
def node_embedding_generate(path, path_bert):
  contents = os.listdir(path)
  l = 0
  """contents2 = os.listdir(path_bert)
  for x in contents2:
    contents.remove(x[:-3]+'csv')"""
  for ar in contents:
    print("remaining : "+str(len(contents)- l))
    l += 1
    path1 = os.path.join(path, ar)
    df = pd.read_csv(path1)
    sentences = list(df['sentences'])

    embedding = []
    for sent in sentences:
      sent = str(sent)
      doc = nlp(sent)
      G = nx.DiGraph()
      for token in doc:
        G.add_node(token.i, label=token.text)
      for token in doc:
        if token.dep_ == "ROOT":
          root_token = token
          idx = token.i
        G.add_edge(token.head.i, token.i, label=token.dep_)
      G = G.reverse()
      node2vec = Node2Vec(G, dimensions=128, walk_length=80, num_walks=10)
      model = node2vec.fit(window=10, min_count=1)
      required_embedding = model.wv.get_vector(idx)
      word_vector = nlp(root_token.text).vector
      res = list(np.concatenate((required_embedding, word_vector), axis=0))
      embedding.append(res)
    name = ar[:-4]+'.npy'
    path_to_npy_file = os.path.join(path_bert, name)
    with open(path_to_npy_file, 'wb') as f:
      np.save(f, np.array(embedding))

def main3(path):
  folders = ['training/annual_reports', 'validation/annual_reports']
  for folder in folders:
    path1 = os.path.join(path, folder)
    path2 = os.path.join(path, folder+'_node_embedding')
    try:
      os.mkdir(path2)
    except:
      res = 0

    node_embedding_generate(path1, path2)
main3('/content/drive/MyDrive/DATA')

Training model1 using bert embeddigs

In [ ]:
def train_bert_model1(path1, path2):
  contents = os.listdir(path1)
  contents_bert = os.listdir(path2)
  x_train = []
  y_train = []
  for i in range(len(contents)):
    print(i)
    x = contents[i]
    y = x[:-3]+'npy'
    path3 = os.path.join(path1, x)
    path4 = os.path.join(path2, y)

    df = pd.read_csv(path3)
    temp2 = list(df['label'])
    temp1 = list(np.load(path4))
    if len(temp1) == len(temp2):
      x_train.extend(temp1)
      y_train.extend(temp2)
    else:
      print("diff length : "+x)
  x_train = np.array(x_train)
  y_train = np.array(y_train)

  scaler = MinMaxScaler()
  scaler.fit(x_train)
  x_train_scaled = scaler.transform(x_train)
  joblib.dump(scaler, '/content/drive/MyDrive/DATA/bert_embedding_scaler.gz')

  X_train = []
  for x in x_train_scaled:
    X_train.append(x.reshape(1, len(x)))
  X_train = np.array(X_train)

  inputs = tf.keras.Input(shape=(1, 770))
  l1 = Bidirectional(LSTM(units=50))(inputs)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(l1)
  model_1 = tf.keras.Model(inputs=inputs, outputs =outputs)

  with open('/content/drive/MyDrive/DATA/lstm_model_bert_embedding_summary.txt', 'w') as f:
    model_1.summary(print_fn=lambda x: f.write(x + '\n'))
  model_1.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

  checkpoint_filepath = '/content/drive/MyDrive/DATA/lstm_model_bert_embedding_weights.h5'
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

  history = model_1.fit(X_train, X_train, epochs=100, batch_size=128, validation_split=0.3, callbacks=[model_checkpoint_callback])
  model_1.load_weights(checkpoint_filepath)
  model_1.save('/content/drive/MyDrive/DATA/bert_embedding_lstm.keras')

  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.savefig('/content/drive/MyDrive/DATA/bert_embedding_lstm_accuracy.png')


  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.savefig('/content/drive/MyDrive/DATA/bert_embedding_lstm_loss.png')

def main(path):
  folders = ['training/annual_reports', 'training/annual_reports_bert_embedding']
  path1 = os.path.join(path, folders[0])
  path2 = os.path.join(path, folders[1])

  train_bert_model1(path1, path2)
main('/content/drive/MyDrive/DATA')

Training model2 using node embeddigs

In [ ]:
def train_node_model2(path1, path2):
  contents = os.listdir(path1)
  contents_bert = os.listdir(path2)
  x_train = []
  y_train = []
  for i in range(len(contents)):
    print(i)
    x = contents[i]
    y = x[:-3]+'npy'
    path3 = os.path.join(path1, x)
    path4 = os.path.join(path2, y)

    df = pd.read_csv(path3)
    temp2 = list(df['label'])
    temp1 = list(np.load(path4))
    if len(temp1) == len(temp2):
      x_train.extend(temp1)
      y_train.extend(temp2)
    else:
      print("diff length : "+x)
  x_train = np.array(x_train)
  y_train = np.array(y_train)
  scaler = MinMaxScaler()
  scaler.fit(x_train)
  x_train_scaled = scaler.transform(x_train)
  joblib.dump(scaler, '/content/drive/MyDrive/DATA/node_embedding_scaler.gz')

  X_train = []
  for x in x_train_scaled:
    X_train.append(x.reshape(1, len(x)))
  X_train = np.array(X_train)
  inputs = tf.keras.Input(shape=(1, 224))
  l1 = Bidirectional(LSTM(units=50))(inputs)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(l1)
  model_1 = tf.keras.Model(inputs=inputs, outputs =outputs)

  with open('/content/drive/MyDrive/DATA/lstm_model_node_embedding_summary.txt', 'w') as f:
    model_1.summary(print_fn=lambda x: f.write(x + '\n'))
  model_1.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

  checkpoint_filepath = '/content/drive/MyDrive/DATA/lstm_model_node_embedding_weights.h5'
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

  history = model_1.fit(X_train, X_train, epochs=100, batch_size=128, validation_split=0.3, callbacks=[model_checkpoint_callback])
  model_1.load_weights(checkpoint_filepath)
  model_1.save('/content/drive/MyDrive/DATA/node_embedding_lstm.keras')

  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.savefig('/content/drive/MyDrive/DATA/node_embedding_lstm_accuracy.png')


  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.savefig('/content/drive/MyDrive/DATA/node_embedding_lstm_loss.png')

def main(path):
  folders = ['training/annual_reports', 'training/annual_reports_node_embedding']
  path1 = os.path.join(path, folders[0])
  path2 = os.path.join(path, folders[1])

  train_node_model2(path1, path2)
main('/content/drive/MyDrive/DATA')

Label prediction model1

In [ ]:
scaler = joblib.load('/content/drive/MyDrive/DATA/bert_embedding_scaler.gz')
model = tf.keras.models.load_model('/content/drive/MyDrive/DATA/bert_embedding_lstm.keras')
path = "/content/drive/MyDrive/DATA/validation/annual_reports"
path_bert = "/content/drive/MyDrive/DATA/validation/annual_reports_bert_embedding"

def predict_labels(x_test, y_test, scalar):
  X_test = []

  x_test = np.array(x_test)
  y_test = np.array(y_test)
  x_test_scaled = scaler.transform(x_test)
  for x in x_test_scaled:
    X_test.append(x.reshape(1, len(x)))
  X_test = np.array(X_test)
  return X_test, y_test


#saving the predictd labels
contents = os.listdir(path)
for i in range(len(contents)):
  print(i)
  x = contents[i]
  y = x[:-3]+'npy'
  path1 = os.path.join(path, x)
  path2 = os.path.join(path_bert, y)

  df = pd.read_csv(path1)
  temp2 = list(df['label'])
  temp1 = np.load(path2)
  if len(temp1) == len(temp2):
    X_test, y_test = predict_labels(temp1, temp2, scaler)
    pred = model.predict(X_test)
    y_pred = []
    for x in pred:
      if x > .3:
        y_pred.append(1)
      else:
        y_pred.append(0)
    print(path1)
    df['predicted label'] = y_pred
    df.to_csv(path1)
  else:
    print("diff length : "+x)

Candidate sequences model1

In [ ]:
import os
import pandas as pd
import numpy as np
import shutil
import tensorflow as tf
import keras
import joblib
model = tf.keras.models.load_model('/content/drive/MyDrive/DATA/bert_embedding_lstm.keras')
scaler_node = joblib.load('/content/drive/MyDrive/DATA/node_embedding_scaler.gz')
scaler_bert = joblib.load('/content/drive/MyDrive/DATA/bert_embedding_scaler.gz')

def predict_labels(x_test, y_test, scaler):
  X_test = []

  X_test = np.array(X_test)
  x_test = np.array(x_test)
  y_test = np.array(y_test)
  x_test_scaled = scaler.transform(x_test)
  for x in x_test_scaled:
    X_test.append(x.reshape(1, len(x)))
  return X_test, y_test

def get_chunks(sent_list):
  sent_length = []
  for sent in sent_list:
    sent = str(sent)
    temp = sent.split(' ')
    sent_length.append(len(temp))
  Seq_indices = []

  start = 0
  step = 0
  count = 0
  while count < len(sent_length):
    if step < 1000:
      step += sent_length[count]
      count += 1
    else:
      stop = count - 1
      Seq_indices.append((start, stop))
      start = count
      step = sent_length[count]
      count += 1

  Seq_indices.append((start, count-1))
  return Seq_indices

def get_text(sent_list, tup):
  a, b = tup[0], tup[1]
  i = a
  text = ''
  while i <= b:
    text += str(sent_list[i])
    if i < b:
      text += ' '
    i += 1
  return text

def get_node_embed_score(X_test, tup):
  a, b = tup[0], tup[1]
  i = a
  score_list = []
  while i <= b:
    temp = X_test[i]
    score_list.append(temp)
    i += 1
  deno = b-a+1
  res = np.sum(score_list)/float(deno)
  return res

def normalize(X_text):
  mean_list = []
  for x in X_text:
    mean_list.append(np.mean(x))
  max_val = np.max(mean_list)
  min_val = np.min(mean_list)
  deno = max_val - min_val
  mean_list = np.array(mean_list)
  mean_list = mean_list - min_val
  min_val /= float(deno)
  return mean_list

node_path = "/content/drive/MyDrive/DATA/validation/annual_reports_node_embedding"
bert_path = "/content/drive/MyDrive/DATA/validation/annual_reports_bert_embedding"
sent_path = "/content/drive/MyDrive/DATA/validation/annual_reports"
save_summary_path = "/content/drive/MyDrive/DATA/SUM_Bert"
try:
  os.mkdir(save_summary_path)
except:
  print("exists!")
#sum_path = "/home/student/Music/fns2020_dataset/node_embedding/vaild_sents"
contents = os.listdir(sent_path)
contents_bert = os.listdir(bert_path)
contents_node = os.listdir(node_path)

x_test = []
pred_dict = {}


for i in range(len(contents)):
  print(i)
  x = contents[i]
  y = x[:-3]+'npy'
  path1 = os.path.join(sent_path, x)
  path2 = os.path.join(bert_path, y)
  path3 = os.path.join(node_path, y)
  if x in contents and y in contents_bert and y in contents_node:
    df = pd.read_csv(path1)
    sents = list(df['sentences'])
    labels = list(df['predicted label'])
    bert_feat = np.load(path2)
    node_feat = np.load(path3)
    bert_score = normalize(bert_feat)
    node_score = normalize(node_feat)
    tfidf_1 = list(df['tf_idf_n_gram_1'])
    tfidf_2 = list(df['tf_idf_n_gram_2'])
    tfidf_3 = list(df['tf_idf_n_gram_3'])
    selected_sents = []
    seq_text = []
    seq_bert_score = []
    seq_node_score = []
    tfidf_1_score = []
    tfidf_2_score = []
    tfidf_3_score = []
    for i in range(len(sents)):
      if labels[i] == 1:
        selected_sents.append(sents[i])
    if len(selected_sents) > 0:
      seq_ids = get_chunks(selected_sents)
      print(seq_ids)
      for tup in seq_ids:
        seq_text.append(get_text(selected_sents, tup))
        seq_bert_score.append(get_node_embed_score(bert_score, tup))
        seq_node_score.append(get_node_embed_score(node_score, tup))
        tfidf_1_score.append(get_node_embed_score(tfidf_1, tup))
        tfidf_2_score.append(get_node_embed_score(tfidf_2, tup))
        tfidf_3_score.append(get_node_embed_score(tfidf_3, tup))
      data = {"summary": seq_text, "bert_score": seq_bert_score, "node_score": seq_node_score, "tf_idf_n_gram_1_score": tfidf_1_score, "tf_idf_n_gram_2_score": tfidf_2_score, "tf_idf_n_gram_3_score": tfidf_3_score}
      df_new = pd.DataFrame(data)
      fin_path = os.path.join(save_summary_path, x)
      df_new.to_csv(fin_path)

Label prediction model2

In [ ]:
scaler = joblib.load('/content/drive/MyDrive/DATA/node_embedding_scaler.gz')
model = tf.keras.models.load_model('/content/drive/MyDrive/DATA/node_embedding_lstm.keras')
path = "/content/drive/MyDrive/DATA/validation/annual_reports"
path_bert = "/content/drive/MyDrive/DATA/validation/annual_reports_node_embedding"

def predict_labels(x_test, y_test, scalar):
  X_test = []

  x_test = np.array(x_test)
  y_test = np.array(y_test)
  x_test_scaled = scaler.transform(x_test)
  for x in x_test_scaled:
    X_test.append(x.reshape(1, len(x)))
  X_test = np.array(X_test)
  return X_test, y_test


#saving the predictd labels
contents = os.listdir(path)
for i in range(len(contents)):
  print(i)
  x = contents[i]
  y = x[:-3]+'npy'
  path1 = os.path.join(path, x)
  path2 = os.path.join(path_bert, y)

  df = pd.read_csv(path1)
  temp2 = list(df['label'])
  temp1 = np.load(path2)
  if len(temp1) == len(temp2):
    X_test, y_test = predict_labels(temp1, temp2, scaler)
    pred = model.predict(X_test)
    y_pred = []
    for x in pred:
      if x > .3:
        y_pred.append(1)
      else:
        y_pred.append(0)
    print(path1)
    df['predicted label'] = y_pred
    df.to_csv(path1)
  else:
    print("diff length : "+x)

Candidate sequences model2

In [ ]:
import os
import pandas as pd
import numpy as np
import shutil
import tensorflow as tf
import keras
import joblib
model = tf.keras.models.load_model('/content/drive/MyDrive/DATA/node_embedding_lstm.keras')
scaler_node = joblib.load('/content/drive/MyDrive/DATA/node_embedding_scaler.gz')
scaler_bert = joblib.load('/content/drive/MyDrive/DATA/bert_embedding_scaler.gz')

def predict_labels(x_test, y_test, scaler):
  X_test = []

  X_test = np.array(X_test)
  x_test = np.array(x_test)
  y_test = np.array(y_test)
  x_test_scaled = scaler.transform(x_test)
  for x in x_test_scaled:
    X_test.append(x.reshape(1, len(x)))
  return X_test, y_test

def get_chunks(sent_list):
  sent_length = []
  for sent in sent_list:
    sent = str(sent)
    temp = sent.split(' ')
    sent_length.append(len(temp))
  Seq_indices = []

  start = 0
  step = 0
  count = 0
  while count < len(sent_length):
    if step < 1000:
      step += sent_length[count]
      count += 1
    else:
      stop = count - 1
      Seq_indices.append((start, stop))
      start = count
      step = sent_length[count]
      count += 1

  Seq_indices.append((start, count-1))
  return Seq_indices

def get_text(sent_list, tup):
  a, b = tup[0], tup[1]
  i = a
  text = ''
  while i <= b:
    text += str(sent_list[i])
    if i < b:
      text += ' '
    i += 1
  return text

def get_node_embed_score(X_test, tup):
  a, b = tup[0], tup[1]
  i = a
  score_list = []
  while i <= b:
    temp = X_test[i]
    score_list.append(temp)
    i += 1
  deno = b-a+1
  res = np.sum(score_list)/float(deno)
  return res

def normalize(X_text):
  mean_list = []
  for x in X_text:
    mean_list.append(np.mean(x))
  max_val = np.max(mean_list)
  min_val = np.min(mean_list)
  deno = max_val - min_val
  mean_list = np.array(mean_list)
  mean_list = mean_list - min_val
  min_val /= float(deno)
  return mean_list

node_path = "/content/drive/MyDrive/DATA/validation/annual_reports_node_embedding"
bert_path = "/content/drive/MyDrive/DATA/validation/annual_reports_bert_embedding"
sent_path = "/content/drive/MyDrive/DATA/validation/annual_reports"
save_summary_path = "/content/drive/MyDrive/DATA/SUM_Node"
try:
  os.mkdir(save_summary_path)
except:
  print("exists!")
#sum_path = "/home/student/Music/fns2020_dataset/node_embedding/vaild_sents"
contents = os.listdir(sent_path)
contents_bert = os.listdir(bert_path)
contents_node = os.listdir(node_path)

x_test = []
pred_dict = {}


for i in range(len(contents)):
  print(i)
  x = contents[i]
  y = x[:-3]+'npy'
  path1 = os.path.join(sent_path, x)
  path2 = os.path.join(bert_path, y)
  path3 = os.path.join(node_path, y)
  if x in contents and y in contents_bert and y in contents_node:
    df = pd.read_csv(path1)
    sents = list(df['sentences'])
    labels = list(df['predicted label'])
    bert_feat = np.load(path2)
    node_feat = np.load(path3)
    bert_score = normalize(bert_feat)
    node_score = normalize(node_feat)
    tfidf_1 = list(df['tf_idf_n_gram_1'])
    tfidf_2 = list(df['tf_idf_n_gram_2'])
    tfidf_3 = list(df['tf_idf_n_gram_3'])
    selected_sents = []
    seq_text = []
    seq_bert_score = []
    seq_node_score = []
    tfidf_1_score = []
    tfidf_2_score = []
    tfidf_3_score = []
    for i in range(len(sents)):
      if labels[i] == 1:
        selected_sents.append(sents[i])
    if len(selected_sents) > 0:
      seq_ids = get_chunks(selected_sents)
      print(seq_ids)
      for tup in seq_ids:
        seq_text.append(get_text(selected_sents, tup))
        seq_bert_score.append(get_node_embed_score(bert_score, tup))
        seq_node_score.append(get_node_embed_score(node_score, tup))
        tfidf_1_score.append(get_node_embed_score(tfidf_1, tup))
        tfidf_2_score.append(get_node_embed_score(tfidf_2, tup))
        tfidf_3_score.append(get_node_embed_score(tfidf_3, tup))
      data = {"summary": seq_text, "bert_score": seq_bert_score, "node_score": seq_node_score, "tf_idf_n_gram_1_score": tfidf_1_score, "tf_idf_n_gram_2_score": tfidf_2_score, "tf_idf_n_gram_3_score": tfidf_3_score}
      df_new = pd.DataFrame(data)
      fin_path = os.path.join(save_summary_path, x)
      df_new.to_csv(fin_path)

Sequence score calculation

In [64]:
import os
import pandas as pd
import numpy as np
from pathlib import Path


params = [(0.5, 0.5, 0, 1), (0.5, 0, 0.5, 1), (0.33, 0.33, 0.33, 1), (0.5, 0.5, 0, 2), (0.5, 0, 0.5, 2), (0.33, 0.33, 0.33, 2), (0.5, 0.5, 0, 3), (0.5, 0, 0.5, 3), (0.33, 0.33, 0.33, 3),
(1, 0, 0, 1), (1, 0, 0, 2), (1, 0, 0, 3)]

summaries = ['/content/drive/MyDrive/DATA/SUM_Bert', '/content/drive/MyDrive/DATA/SUM_Node']
project_sys = ['/content/drive/MyDrive/DATA/TIBER_1_Bert_Model', '/content/drive/MyDrive/DATA/TIBER_2_Node_Model']
project_ref = ['system', 'reference']
for x in project_sys:
  try:
    os.mkdir(x)
    for y in project_ref:
      os.mkdir(os.path.join(x, y))
  except:
    res = 0

alpha = []
beta = []
gamma = []
tfidf_n_gram = []
sys = []
for i in range(len(summaries)):
  summ = summaries[i]
  contents1 = os.listdir(summ)
  for k in range(len(params)):
    param = params[k]
    alpha.append(param[0])
    beta.append(param[1])
    gamma.append(param[2])
    tfidf_n_gram.append(param[3])
    sys.append(k+1)
    for j in range(len(contents1)):
     try:
      folder_path = os.path.join(summ, contents1[j])
      df = pd.read_csv(folder_path)
      sentences = list(df["summary"])
      bert_score = list(df["bert_score"])
      node_score = list(df["node_score"])
      tfidf = list(df["tf_idf_n_gram_"+str(param[3])+"_score"])
      scores = param[0]*np.array(tfidf) + param[1]*np.array(bert_score) + param[2]*np.array(node_score)
      #print(param, np.max(scores), np.argmax(scores), len(scores))
      #print(scores)
      id_text = np.argmax(scores)
      text = sentences[id_text]
      text_name = contents1[j][:-4]+'_system'+str(k+1)+'.txt'
      text_path = os.path.join(project_sys[i], 'system')
      text_path = os.path.join(text_path, text_name)
      Path(text_path).touch()
      text_ptr = open(text_path, 'w')
      text_ptr.write(text)
      text_ptr.close()
     except:
      res = 0
data = {'alpha':alpha, 'beta':beta, 'gamma':gamma, 'tfidf_n_gram':tfidf_n_gram, 'system file':sys}
df_data = pd.DataFrame(data)
df_data.to_csv('/content/drive/MyDrive/DATA/param_to_system_file_mapping.csv')

Renaming gold summaries as reference

In [65]:
path_val_gs = '/content/drive/MyDrive/DATA/validation/gold_summaries'
path_vsl_ref = ['/content/drive/MyDrive/DATA/TIBER_1_Bert_Model/reference', '/content/drive/MyDrive/DATA/TIBER_2_Node_Model/reference']
contents = os.listdir(path_val_gs)
unique_files = []
for x in contents:
  unique_files.append(x.split('_')[0])
unique_files = list(set(unique_files))
for x in unique_files:
  i = 1
  for y in contents:
    if y.split('_')[0] == x:
      src_path = os.path.join(path_val_gs,y)
      name = x+'_reference'+str(i)+'.txt'
      dest_path1 = os.path.join(path_vsl_ref[0],name)
      dest_path2 = os.path.join(path_vsl_ref[1],name)
      shutil.copy(src_path, dest_path1)
      shutil.copy(src_path, dest_path2)
      i += 1

Rouge score calculation

In [ ]:
!git clone https://github.com/kavgan/ROUGE-2.0.git


In [ ]:
!unzip '/content/ROUGE-2.0/versions/v1.2.2/rouge2_v1.2.2_runnable.zip'

In [69]:
#TIBER1
try:
    shutil.rmtree('/content/v1.2.2/projects/test-summarization/reference')
    shutil.rmtree('/content/v1.2.2/projects/test-summarization/system')
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))
shutil.copytree('/content/drive/MyDrive/DATA/TIBER_1_Bert_Model/system','/content/v1.2.2/projects/test-summarization/system')
shutil.copytree('/content/drive/MyDrive/DATA/TIBER_1_Bert_Model/reference','/content/v1.2.2/projects/test-summarization/reference')
%cd '/content/v1.2.2'
subprocess.call(["java","-jar","rouge2-1.2.2.jar"])
shutil.copy('/content/v1.2.2/results.csv','/content/results_model1.csv')

/content/v1.2.2


'/content/results_model1.csv'

In [70]:
#TIBER2
try:
    shutil.rmtree('/content/v1.2.2/projects/test-summarization/reference')
    shutil.rmtree('/content/v1.2.2/projects/test-summarization/system')
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))
shutil.copytree('/content/drive/MyDrive/DATA/TIBER_1_Bert_Model/system','/content/v1.2.2/projects/test-summarization/system')
shutil.copytree('/content/drive/MyDrive/DATA/TIBER_1_Bert_Model/reference','/content/v1.2.2/projects/test-summarization/reference')
%cd '/content/v1.2.2'
subprocess.call(["java","-jar","rouge2-1.2.2.jar"])
shutil.copy('/content/v1.2.2/results.csv','/content/results_model2.csv')

/content/v1.2.2


'/content/results_model2.csv'